In [2]:
import os 
import time
import requests
import json
import re
import pandas as pd
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool


indexnum=0

#list of [片名，路径]
namelist=[]

#已下载无字幕番号合集
nonsublist=[]

def sortFunc(e):
    return e[1]

def get_size(start_path = '.'):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return (total_size/1024/1024)



def largets_file(start_path = '.'):
    largest=''
    max_size=0
    for f in next( os.walk(start_path) )[2]:
        fp = os.path.join(start_path, f)
        size=os.path.getsize(fp)
        if size>max_size:
            max_size=size
            largest=fp
    return largest

def largets_file_size(start_path = '.'):
    #print(start_path)
    max_size=0
    if len(next( os.walk(start_path) )[2])>0:
        for f in next( os.walk(start_path) )[2]:
            fp = os.path.join(start_path, f)
            size=os.path.getsize(fp)
            if size>max_size:
                max_size=size
    return max_size/1024/1024


headers = {'User-Agent':
               'Mozilla/5.0 (Windows NT 10.0; WOW64)\
               AppleWebKit/537.36(KHTML, like Gecko)\
               Chrome/55.0.2883.87 Safari/537.36'}


def findmag(prefixnum):
    prefix=prefixnum[0]
    num=prefixnum[1]
    result=[]
    print(prefix.upper()+'-'+num+'开始搜索:')
    print()
    maglist=[]
    keywordlist=[]
    
    keywordlist.append(prefix+'-'+num+'r')
    keywordlist.append(prefix+'-'+num+'c')
    keywordlist.append(prefix+num+'c')
    keywordlist.append(prefix+num+'cmp4')
    
    for keyword in keywordlist:
        url = 'https://m.zhongziso.com/list/'+keyword+'/1'

        web = requests.get(url, headers=headers)
        web.encoding = 'utl-8'

        Page = web.text

        indexr=Page.find('<span class="highlight">'+keyword+'</span>')

        if (indexr!=-1):
            print(keyword+'有带字幕链接')
            result.append(keyword+'有带字幕链接')
            result.append('网址：')
            result.append(url)
            result.append('磁力链接：')

            maglist=re.findall('magnet:/?.*?(?=")', Page)
            for item in maglist:
                result.append(item)
            result.append(' ')
    
    print('-------------------------------------------------------------------------')
    print()
    return result
        
def findactress(code):

    url = 'https://www.javbus.com/'+code

    web = requests.get(url, headers=headers)
    web.encoding = 'utl-8'

    Page = web.text
    #<div class="star-name"><a href="https://www.javbus.com/star/2xi" title="葵つかさ">葵つかさ</a></div>

    actresslist=re.findall(r'<div class="star-name"><a href="https://www.javbus.com/star/.*?>(.*?)<', Page)

    return [code,actresslist]

# 所有子文件夹的女优

In [33]:
#需找演员的番号列表

codelist=[]


drivelist=[]
#drivelist.append(os.walk(r"g:/renamed"))
#drivelist.append(os.walk(r"g:/BEST1"))
drivelist.append(os.walk(r"g:/Checked"))
#drivelist.append(os.walk(r"g:/Collected"))

#drivelist.append(os.walk(r"i:/Checked (Non Subbed)/"))

collectedlist=[]
for dir_name in next(os.walk('g:/Collected/'))[1]:
    collectedlist.append(dir_name)

for g in drivelist:
    for path,dir_list,file_list in g:  
        for dir_name in dir_list:
            index_=dir_name.find('-')
            if (index_!=-1) and (index_<len(dir_name)-1):
                indexnum=index_+1
                while (dir_name[indexnum].isdigit()) and (indexnum<len(dir_name)):
                    indexnum+=1
                    if indexnum==len(dir_name):
                        break
                code=dir_name[:indexnum]

                codelist.append(code)

codelist.sort()

pool = ThreadPool()
results=pool.map(findactress,codelist)
results.sort(key=sortFunc)


In [40]:
tocollect=[]
for result in results:
    if len(result[1])>0:
        for name in result[1]:
            for collectedname in collectedlist:
                if collectedname.find(name)!=-1:
                    tocollect.append([result[0],collectedname,result[1]])


In [43]:
for x in tocollect:
    for item in x:
        print(item)
    print()

SDDE-500
AIKA
['AIKA', 'MIRANO', '月嶋あかり']

ZUKO-088
AIKA
['AIKA', '松本メイ', '阿部乃みく', '一条リオン']

KIBD-232
AIKA
['AIKA', '緒川りお', '若菜奈央', '椎名そら', '麻里梨夏', '姫川ゆうな', '千夏まりな', '天野美優', '愛瀬美希', '桜庭このみ', 'SARA']

KIBD-232
椎名そら_节选
['AIKA', '緒川りお', '若菜奈央', '椎名そら', '麻里梨夏', '姫川ゆうな', '千夏まりな', '天野美優', '愛瀬美希', '桜庭このみ', 'SARA']

HAR-029
あおいれな（葵玲奈）
['あおいれな', '吹石れな', '日比乃さとみ', '大沢莉奈']

AVOP-382
跡美しゅり
['つぼみ', '跡美しゅり', '向井藍', 'あず希', '宮沢ゆかり', '山川ゆな']

EBOD-511
NAOMI
['ティア', 'NAOMI']

IPZ-948
椎名そら_节选
['ティア', '椎名そら']

RTP-060
明日美さき
['七菜原ココ', '明日美さき', '並木あゆ']

GNE-158
上原瑞穂
['上原瑞穂', '川村まや', '美里有紗', '松嶋葵', '桐嶋りの']

IPZ-757
希美まゆ
['天海つばさ', '希美まゆ']

DOCP-069
芽森しずく（宝生リリー）
['宝生リリー', 'みゆき菜々子', '白咲りの', '新垣ふみ']

ZUKO-104
椎名そら_节选
['川村まや', '椎名そら', '渡辺そら', '鳥井真央']

MIAE-084
椎名そら_节选
['椎名そら', '麻里梨夏']

SDDE-401
市川まさみ
['水希杏', '手嶋ゆな', '竹内真琴', '舞園かりん', '涼南佳奈', '浅倉あすか', '春海紗奈', '市川まさみ', '陽木かれん']

SDMU-842
星奈あい
['水野朝陽', '西宮このみ', '星奈あい']

MIAE-188
AIKA
['波多野結衣', 'AIKA']

WANZ-769
椎名そら_节选
['波多野結衣', '篠田ゆう', '北川エリカ', '神波多一花', '浜崎真緒', '椎名そ

In [29]:
for dir_name in next(os.walk('g:/checked'))[1]:
    for x in tocollect:
        if dir_name.find(x[0])!=-1:
            path='g:/checked/'+dir_name
            dst='g:/collected/'+x[1]+'/'+dir_name
            os.rename(path,dst)

# Collected文件夹检查

In [11]:
drivelist=[]

root=r"g:/Collected"
for dir_name in next( os.walk(root) )[1]:
    drivelist.append(dir_name)

for actressname in drivelist:
    print('------------------------------------------')
    print(actressname)
    path=os.path.join(root, actressname)
    g=os.walk(path)
    codelist=[]
    for path,dir_list,file_list in g:  
        
        for dir_name in dir_list:
            index_=dir_name.find('-')
            if (index_!=-1) and (index_<len(dir_name)-1):
                indexnum=index_+1
                while (dir_name[indexnum].isdigit()) and (indexnum<len(dir_name)):
                    indexnum+=1
                    if indexnum==len(dir_name):
                        break
                code=dir_name[:indexnum]

                codelist.append(code)

    codelist.sort()

    pool = ThreadPool()
    results=pool.map(findactress,codelist)
    results.sort(key=sortFunc)
    for result in results:
        if len(result[1])>0 and actressname not in result[1]:
            print()
            for line in result:
                print(line)

------------------------------------------
Julia

MIDD-994
['JULIA']
------------------------------------------
NAOMI
------------------------------------------
RION

SNIS-166
['宇都宮しをん']
------------------------------------------
sex friend
------------------------------------------
あおいれな（葵玲奈）

MIAD-911
['あおいれな']

MIAE-046
['あおいれな']
------------------------------------------
あべみかこ（安部未华子）

BKKG-014
['あべみかこ']

CWM-241
['あべみかこ']

HND-218
['あべみかこ']

HND-282
['あべみかこ']

MIAD-758
['あべみかこ']

MIAD-779
['あべみかこ']

MIAD-956
['あべみかこ']

MIAE-017
['あべみかこ']

MIGD-649
['あべみかこ']

MIGD-699
['あべみかこ']

MIGD-736
['あべみかこ']

MKMP-138
['あべみかこ']

NHDTA-929
['あべみかこ']

WANZ-468
['あべみかこ']

WANZ-521
['あべみかこ']

WANZ-540
['あべみかこ']

MIAD-977
['あべみかこ', '椎名そら']

ZUKO-121
['あべみかこ', '相原翼', '北川ゆず', '麻里梨夏']

MIGD-702
['上原亜衣', 'あべみかこ']
------------------------------------------
あやみ旬果
------------------------------------------
さくらゆら（樱由罗）

KAWD-756
['さくらゆら']

KAWD-775
['さくらゆら']

KAWD-785
['さくらゆら']

EBOD-227
['ちとせりこ']
---------